Code for classifying videos based on emotions present

Currently implemented:
- Mean
- Mode

TODO:
- take in a CNN model
- setup code for mean/mode
- classify on video
- view results

In [1]:
from tensorflow.keras.models import load_model
from collections import deque
import numpy as np
from sklearn.preprocessing import LabelBinarizer
import argparse
import pickle
import cv2

In [2]:
MODEL = "RN50"

if MODEL == "RN50":
    model = load_model('../models/best-models/resnet50.h5')


In [18]:
lb = pickle.loads(open("label_bin", "rb").read())

mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")
size = 2
Q = deque(maxlen=size)



In [19]:
video_fn = "../data/dataset/02-01-02-01-18-30-720.mp4"
output_fn = "results/test.mp4"

vs = cv2.VideoCapture(video_fn)
writer = None
(W, H) = (None, None)

# Loop over video frames
while True:
    (grabbed, frame) = vs.read()
    if not grabbed:
        break
        
    if W is None or H is None:
        (H, W) = frame.shape[:2]
        
    output = frame.copy()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (224, 224)).astype("float32")
    frame -= mean
    
    preds = model.predict(np.expand_dims(frame, axis=0))[0]
    Q.append(preds)
    
    results = np.array(Q).mean(axis=0)
    i = np.argmax(results)
    label = lb.classes_[i]
    
    text = "emotion : {}".format(label)
    cv2.putText(output, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.25, (0, 255, 0), 5)
    
    if writer is None:
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        # TODO: setup fps?
        writer = cv2.VideoWriter(output_fn, fourcc, 30, (W,H), True)
        
    writer.write(output)
    cv2.imshow("Output", output)
    key = cv2.waitKey(1) & 0xFF
    
    
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

cv2.destroyWindow('Output')

writer.release()
vs.release()

    